In [1]:
import numpy as np
import networkx as nx
from SNN_neat import DAG, plot_dag
from SNN_neat import ReLU, LeakyReLU, Sigmoid, Linear, Tanh
from tqdm import tqdm
import plotly.graph_objects as go
from time import time

In [2]:
WIDTH = 100
NET_DEPH = 5

dag = DAG(WIDTH, WIDTH)

for i, node_id in enumerate(dag.input_nodes):
    out_node = dag.output_nodes[i]
    dag.add_connection(node_id, out_node)
    
    # add node in input connection
    new_node = node_id
    for i in range(NET_DEPH):
        _, new_node = dag.add_node(new_node, out_node)

dag.processing_order = dag.get_processing_order()
for i, layer in enumerate(dag.processing_order[:-1]):
    for node1 in layer:
        for node2 in dag.processing_order[i+1]:
            dag.add_connection(node1, node2)

dag.processing_order = dag.get_processing_order()
print(dag)

DAG(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
	SpikingNode(0: bias: -0.522712209278368, threshold: 0.7, in: [] out: ['200', '205', '210', '215', '220', '225', '230', '235', '240', '245', '250', '255', '260', '265', '270', '275', '280', '285', '290', '295', '300', '305', '310', '315', '320', '325', '330', '335', '340', '345', '350', '355', '360', '365', '370', '375', '380', '385', '390', '395', '400', '405', '410', '415', '420', '425', '430', '435'

In [3]:
def calculate_sigma(w, b, sigma=1, threshold=1):
    if len(w) == 0:
        return w, b
    
    pm = np.where(w > 0, w, 0)
    nm = np.where(w < 0, w, 0)

    ps = (np.sum(pm) + (b > 0) * b)
    ns = - (np.sum(nm) + (b < 0) * b)

    if ps == 0:
        # flip ranfom positive weight
        i = np.random.randint(0, len(w))
        w[i] = abs(w[i])
        pm = np.where(w > 0, w, 0)
        ps = np.sum(pm)

    r = ((sigma) - (ps - ns)) / len(w + (b != 0))
    return w + r, (b + r) * (b != 0)

sigma = 0.123
w, b = calculate_sigma(np.array([1, 1, 1, 1]), 0, sigma=sigma, threshold=1)
print(w, round(np.sum(w), 3), round(np.sum(w), 3)  == sigma)

[0.03075 0.03075 0.03075 0.03075] 0.123 True


In [4]:
THRESHOLD = 0.5
PERCENTAGE = 0.3
ITERATIONS = 100

results = []

for sigma in tqdm(np.arange(-10, 5, 0.5), desc="Sigma"):
    # set threshold
    for layer in dag.processing_order:
        for nodeId in layer:
            node = dag.get_node(nodeId)
            node.threshold = THRESHOLD

    output_avg_activations = []
    # iterate multipple times
    real_percentages = []
    for _ in range(ITERATIONS):
        dag.reset()
        inputs = [int(np.random.rand() < PERCENTAGE) for _ in range(WIDTH)]

        real_percentages.append(np.sum(inputs) / WIDTH)

        for layer in dag.processing_order:
            for nodeId in layer:
                node = dag.get_node(nodeId)
                node.bias = (np.random.rand() * 2 - 1)
                node.weights = np.random.rand(len(node.weights)) * 2 - 1  
                node.weights, node.bias = calculate_sigma(w=node.weights, b=node.bias, sigma=sigma, threshold=THRESHOLD)

        # process the network #####
        dag.process_once(inputs)
        ###########################

        # get activation 
        avg_activation_probability = 0
        for node_id in dag.output_nodes:
            node = dag.get_node(node_id)
            avg_activation_probability += node.spike

        avg_activation_probability /= len(dag.output_nodes)

        output_avg_activations.append(avg_activation_probability)

    results.append([sigma, np.min(real_percentages), np.average(real_percentages), np.max(real_percentages), np.min(output_avg_activations), np.average(output_avg_activations), np.max(output_avg_activations)])
        
results = np.array(results)

Sigma: 100%|██████████| 30/30 [01:48<00:00,  3.60s/it]


In [5]:
# plot all the tested thresholds, by different lines in the xs vs. ys plot with plotly
# x values are input activity, y values are mean activity of the second layer
fig = go.Figure()
#fig.add_trace(go.Scatter(x=results[:,0], y=results[:,1], mode='lines+markers', name="Min real %"))
fig.add_trace(go.Scatter(x=results[:,0], y=results[:,2], mode='lines+markers', name="Avg real %"))
#fig.add_trace(go.Scatter(x=results[:,0], y=results[:,3], mode='lines+markers', name="Max real %"))
#fig.add_trace(go.Scatter(x=results[:,0], y=results[:,4], mode='lines+markers', name="Min activity"))
fig.add_trace(go.Scatter(x=results[:,0], y=results[:,5], mode='lines+markers', name="Avg activity"))
#fig.add_trace(go.Scatter(x=results[:,0], y=results[:,6], mode='lines+markers', name="Max activity"))

fig.update_xaxes(title="Sigma")
fig.update_yaxes(title="Mean activity")
fig.show()
